<a href="https://colab.research.google.com/github/abolima/face_recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install cmake

In [0]:
!pip install opencv-python

In [0]:
!pip install dlib

In [0]:
!pip install face_recognition

    100% |████████████████████████████████| 100.2MB 383kB/s 
  Running setup.py bdist_wheel for face-recognition-models ... - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [0]:
import numpy as np
import face_recognition
import cv2 as cv2 #OpenCV
import cv2 as cv21#OpenCV
from os import listdir
import time
import threading

In [0]:
facecascade = cv2.CascadeClassifier('e:/haarcascade_frontalface_default.xml')

In [4]:
import json

#put your employee pictures in this path as name_of_employee.jpg
employee_pictures = "e:/employeephotos/"

known_face_encodings =[]
known_face_names =[]

for file in listdir(employee_pictures):
    employee, extension = file.split(".")
    img = face_recognition.load_image_file('e:/employeephotos/%s.jpg' % (employee))
    face_location = face_recognition.face_locations(img ,number_of_times_to_upsample=0, model="cnn")
    img_encoding = face_recognition.face_encodings(img)[0]
    known_face_encodings.append(img_encoding)
    known_face_names.append(employee)


print('employee representations retrieved successfully')      

employee representations retrieved successfully


In [0]:
import threading

class VideoCaptureAsync:
    def __init__(self, src=0, width=640, height=480):
        self.src = src
        self.cap = cv2.VideoCapture(self.src)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        self.grabbed, self.frame = self.cap.read()
        self.started = False
        self.read_lock = threading.Lock()

    def set(self, var1, var2):
        self.cap.set(var1, var2)

    def start(self):
        if self.started:
            print('[!] Asynchroneous video capturing has already been started.')
            return None
        self.started = True
        self.thread = threading.Thread(target=self.update, args=())
        self.thread.start()
        return self

    def update(self):
        while self.started:
            grabbed, frame = self.cap.read()
            with self.read_lock:
                self.grabbed = grabbed
                self.frame = frame

    def read(self):
        with self.read_lock:
            frame = self.frame.copy()
            grabbed = self.grabbed
        return grabbed, frame

    def stop(self):
        self.started = False
        self.thread.join()
        self.cap.release()

    def __exit__(self, exec_type, exc_value, traceback):
        self.cap.release()
    

In [0]:
def rescale_frame(frame, percent=75):
      width = int(frame.shape[1] * percent/ 100)
      height = int(frame.shape[0] * percent/ 100)
      dim = (width, height)
      return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA) 

In [0]:
import requests 
from IPython.core.display import display, HTML

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
#video_capture = cv2.VideoCapture(0)
video_capture = VideoCaptureAsync(0)
video_capture.start()
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    #frame = rescale_frame(frame,50)
      
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]
    
    
    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame ,number_of_times_to_upsample=0, model="cnn")
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding,0.45)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
           

            
        # Draw a box around the face
        cv21.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv21.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv21.FONT_HERSHEY_DUPLEX
        cv21.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv21.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv21.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
#video_capture.release()
video_capture.stop()
cv21.destroyAllWindows()